<a href="https://colab.research.google.com/github/xvmon234-ai/Learning-Python/blob/main/Financial_Risk_Prediction/02_Financial_Ratio_Calculation/Financial_Ratio_Calculation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **재무 비율 계산 (Financial Ratio Calculation)**
이 단계는 재무제표 데이터를 기반으로 주요 재무 비율을 계산하고, 분석에 활용할 수 있는 형태로 정리하는 과정입니다. 이전 단계에서 수행했던 데이터 수집 작업을 포함하여, 하나의 Colab 노트북에서 전체 과정을 진행합니다.


---

#### **1단계: 데이터 수집 및 전처리**
새로운 Colab 노트북에서 작업을 시작하므로, 먼저 이전 단계의 데이터 수집 및 전처리 과정을 반복해야 합니다. 이 코드를 통해 재무 비율 계산에 필요한 데이터를 확보합니다.


In [4]:
import yfinance as yf
import pandas as pd

# 분석 대상 기업 티커 리스트
# 자세한 사항은 01_Data_Acquisition_Preprocessing.ipynb 참고
tickers = ['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'META']

# 데이터 저장용 딕셔너리
financial_data = {}

for ticker in tickers:
  try:
    stock = yf.Ticker(ticker)

    # 재무제표 데이터 불러오기
    income_stmt = stock.income_stmt.T # 손익계산서
    balance_sheet = stock.balance_sheet.T # 재무상태표

    financial_data[ticker] = {
        'income_stmt': income_stmt,
        'balance_sheet': balance_sheet
    }

    print(f"{ticker}의 재무 데이터를 성공적으로 불러왔습니다.")

  except Exception as e:
        print(f"{ticker}의 데이터를 불러오는 중 오류가 발생했습니다: {e}")


AAPL의 재무 데이터를 성공적으로 불러왔습니다.
MSFT의 재무 데이터를 성공적으로 불러왔습니다.
GOOGL의 재무 데이터를 성공적으로 불러왔습니다.
AMZN의 재무 데이터를 성공적으로 불러왔습니다.
META의 재무 데이터를 성공적으로 불러왔습니다.


#### **단계 2: 재무 비율 계산**
이제 수집된 데이터를 바탕으로 재무 비율을 계산합니다. 각 재무 비율의 의미를 주석으로 명시하여 코딩과 회계 지식을 동시에 학습합니다.

In [29]:
# 계산 결과를 저장할 별도의 딕셔너리
calculated_ratios = {}

for ticker in tickers:
  if ticker in financial_data:
    income_df = financial_data[ticker]['income_stmt']
    balance_df = financial_data[ticker]['balance_sheet']

    # DataFrame 복사본을 생성하여 오류 방지
    income_df = income_df.copy()
    balance_df = balance_df.copy()

    # 각 기업별 재무 비율 계산
    ratios_df = pd.DataFrame(index=income_df.index)
    # 질문 05. index 설정에 대한 추가 설명을 부탁해

    # 1. 유동성 비율 (Liquidity Ratios)
    # 유동비율 = 유동자산 / 유동부채: 단기 채무 지급 능력을 나타냄.
    ratios_df['CurrentRatio'] = balance_df['Current Assets'] / balance_df['Current Liabilities']

    # 당좌비율 = (유동자산 - 재고자산) / 유동부채: 재고를 제외한 엄격한 단기 지급 능력.
    # META의 경우 재고자산이 없으므로 재고자산을 0으로 간주하여 계산해야 함
    # 따라서 다음의 조건문을 활용
    if 'Inventory' in balance_df.columns:
      ratios_df['QuickRatio'] = (balance_df['Current Assets'] - balance_df['Inventory']) / balance_df['Current Liabilities']
    # 재고자산이 없는 경우
    # 업종에 따라서 이와 같은 경우가 존재할 수 있음을 유의해야함
    else:
      ratios_df['QuickRatio'] = balance_df['Current Assets'] / balance_df['Current Liabilities']

    # 2. 수익성 비율 (Profitability Ratios)
    # 매출총이익률 = 매출총이익 / 매출(OperatingRevenue): 주된 영업활동을 통해 얻은 수익성.
    ratios_df['GrossProfitMargin'] = income_df['Gross Profit'] / income_df['Operating Revenue']

    # 영업이익률 = 영업이익 / 매출(Operating Revenue): 핵심 사업의 효율성.
    ratios_df['OperatingProfitMargin'] = income_df['Operating Income'] / income_df['Operating Revenue']

    # 3. 레버리지 비율 (Leverage Ratios)
    # 부채비율 = 총부채 / 총자본(Common Stock Equity): 타인자본 의존도를 나타냄.
    ratios_df['DebtRatio'] = balance_df['Total Debt'] / balance_df['Common Stock Equity']

    # 계산된 재무 비율 저장
    calculated_ratios[ticker] = ratios_df
    print(f"{ticker}의 재무 비율 계산 완료.")

# 모든 기업의 계산된 재무 비율을 하나의 DataFrame으로 통합
all_ratios_df = pd.concat(calculated_ratios, names=['Ticker', 'Date'])
all_ratios_df = all_ratios_df.reset_index()

print("\n 모든 기업의 재무 비율 데이터가 통합되었습니다.")
print(all_ratios_df.head())



AAPL의 재무 비율 계산 완료.
MSFT의 재무 비율 계산 완료.
GOOGL의 재무 비율 계산 완료.
AMZN의 재무 비율 계산 완료.
META의 재무 비율 계산 완료.

 모든 기업의 재무 비율 데이터가 통합되었습니다.
  Ticker       Date  CurrentRatio  QuickRatio  GrossProfitMargin  \
0   AAPL 2024-09-30      0.867313    0.826007           0.462063   
1   AAPL 2023-09-30      0.988012    0.944442           0.441311   
2   AAPL 2022-09-30      0.879356    0.847235           0.433096   
3   AAPL 2021-09-30      1.074553    1.022115           0.417794   
4   AAPL 2020-09-30           NaN         NaN                NaN   

   OperatingProfitMargin  DebtRatio  
0               0.315102   1.872327  
1               0.298214   1.787533  
2               0.302887   2.614462  
3               0.297824   2.163925  
4                    NaN        NaN  


> **오류 1: 최초 작업 진행시, `KeyError: 'CurrentAssets'` 발생**
>
> 해당 사항은 기업마다 각 계정명 등이 다르기 때문에 종종 발생할 수 있는 **실무적인 문제**로 이를 해결하기 위한 전반적인 과정을 별도로 기록하여 비슷한 오류를 발견했을 때, 해결하는 역량의 증진에 도움이 되고자 합니다.


---


#### **방법1: 실제 컬럼 이름 확인하기**


In [8]:
# AAPL 기업의 재무상태표 컬럼 확인 예시
# 다른 기업도 동일한 방법으로 확인 가능
balance_sheet_aapl = financial_data['AAPL']['balance_sheet']
print(balance_sheet_aapl.columns)

Index(['Treasury Shares Number', 'Ordinary Shares Number', 'Share Issued',
       'Net Debt', 'Total Debt', 'Tangible Book Value', 'Invested Capital',
       'Working Capital', 'Net Tangible Assets', 'Capital Lease Obligations',
       'Common Stock Equity', 'Total Capitalization',
       'Total Equity Gross Minority Interest', 'Stockholders Equity',
       'Gains Losses Not Affecting Retained Earnings',
       'Other Equity Adjustments', 'Retained Earnings', 'Capital Stock',
       'Common Stock', 'Total Liabilities Net Minority Interest',
       'Total Non Current Liabilities Net Minority Interest',
       'Other Non Current Liabilities', 'Tradeand Other Payables Non Current',
       'Long Term Debt And Capital Lease Obligation',
       'Long Term Capital Lease Obligation', 'Long Term Debt',
       'Current Liabilities', 'Other Current Liabilities',
       'Current Deferred Liabilities', 'Current Deferred Revenue',
       'Current Debt And Capital Lease Obligation',
       'Current C

> 하지만, 해당 방법을 이용하면 대량의 데이터가 발생하여 원하는 데이터만을 식별하기에는 어렵고, 각 기업마다 모든 칼럼을 print해야한다는 점에서 비효율적이라고 판단하여, AI를 활용해 2가지 요구사항을 충족할 수 있는 코딩을 산출해냈습니다.
>
> 요구사항 01. 'Current'가 들어간 계정만을 추출
>
> 요구사항 02. 전체 기업을 한 번에 확인할 수 있게끔
>
> 위의 요구사항을 반영한 방법은 다음과 같습니다.

In [9]:
for ticker, data in financial_data.items():
    balance_sheet = data['balance_sheet']

    # .str.contains()를 이용해 'current'가 포함된 컬럼만 필터링
    current_columns = [col for col in balance_sheet.columns if 'current' in str(col).lower()]

    print(f"✅ {ticker}의 'current' 관련 컬럼:")
    print(current_columns)
    print("-" * 30)

✅ AAPL의 'current' 관련 컬럼:
['Total Non Current Liabilities Net Minority Interest', 'Other Non Current Liabilities', 'Tradeand Other Payables Non Current', 'Current Liabilities', 'Other Current Liabilities', 'Current Deferred Liabilities', 'Current Deferred Revenue', 'Current Debt And Capital Lease Obligation', 'Current Capital Lease Obligation', 'Current Debt', 'Other Current Borrowings', 'Total Non Current Assets', 'Other Non Current Assets', 'Non Current Deferred Assets', 'Non Current Deferred Taxes Assets', 'Current Assets', 'Other Current Assets']
------------------------------
✅ MSFT의 'current' 관련 컬럼:
['Total Non Current Liabilities Net Minority Interest', 'Other Non Current Liabilities', 'Tradeand Other Payables Non Current', 'Non Current Deferred Liabilities', 'Non Current Deferred Revenue', 'Non Current Deferred Taxes Liabilities', 'Current Liabilities', 'Other Current Liabilities', 'Current Deferred Liabilities', 'Current Deferred Revenue', 'Current Debt And Capital Lease Obliga

> 다음의 분석을 통해 `KeyError: 'CurrentAssets'`은 중간의 공백 여부에 의한 것임을 파악해냈습니다. 위의 제시된 코딩은 이와 같은 **오류를 수정한 최종본**임을 알려드립니다.


---

아래는 위와 비슷한 `KeyError`들을 해결한 사례들입니다.

In [23]:
# 수정 내용: GrossProfit > Gross Profit

for ticker, data in financial_data.items():
    income_stmt = data['income_stmt']

    # .str.contains()를 이용해 'gross'가 포함된 컬럼만 필터링
    current_columns = [col for col in income_stmt.columns if 'gross' in str(col).lower()]

    print(f"✅ {ticker}의 'gross' 관련 컬럼:")
    print(current_columns)
    print("-" * 30)

✅ AAPL의 'gross' 관련 컬럼:
['Gross Profit']
------------------------------
✅ MSFT의 'gross' 관련 컬럼:
['Gross Profit']
------------------------------
✅ GOOGL의 'gross' 관련 컬럼:
['Gross Profit']
------------------------------
✅ AMZN의 'gross' 관련 컬럼:
['Gross Profit']
------------------------------
✅ META의 'gross' 관련 컬럼:
['Gross Profit']
------------------------------


최초에 Gross Profit이 Gross PPE 인 줄 알고 잘못 계산한 것에 대한 부분을 교정하기 위해 기록을 남깁니다.

> ### **추가 학습: Gross PP&E란 무엇인가요?**
>
> **Gross PP&E**는 **Gross Property, Plant, and Equipment**의 약어로, 우리말로 **총유형자산**을 의미하는 회계 계정입니다.
>
> 이 계정은 기업이 영업 활동을 위해 보유하고 있는 토지, 건물, 기계장치 등 장기간 사용될 유형자산의 **취득원가(Historical Cost)** 총액을 나타냅니다. 가장 중요한 점은 아직 **감가상각비를 차감하기 전의 금액**이라는 것입니다.
>
> **Gross PP&E와 Net PP&E의 차이**
>
> 회계 분석 시에는 Gross PP&E와 Net PP&E를 구분하는 것이 중요합니다.
>
> * **Gross PP&E**: 자산을 처음 취득했을 때의 원가로, 기업의 **총 투자 규모**를 파악하는 데 유용합니다.
> * **Net PP&E**: Gross PP&E에서 **감가상각누계액(Accumulated Depreciation)**을 차감한 금액으로, 현재 시점에서 자산의 장부상 가치(Book Value)를 나타냅니다.
>
> $$Net PP\&E = Gross PP\&E - Accumulated Depreciation$$
>
> 따라서 Gross PP&E는 기업의 투자 현황을, Net PP&E는 현재 남아있는 자산의 가치를 평가하는 데 사용됩니다.


In [18]:
# 수정 내용: OperatingRevenue > Operating Revenue
# 수정 내용: OperatingIncome > Operating Income

for ticker, data in financial_data.items():
    income_stmt = data['income_stmt']

    # .str.contains()를 이용해 'operating'가 포함된 컬럼만 필터링
    current_columns = [col for col in income_stmt.columns if 'operating' in str(col).lower()]

    print(f"✅ {ticker}의 'operating' 관련 컬럼:")
    print(current_columns)
    print("-" * 30)

✅ AAPL의 'revenue' 관련 컬럼:
['Total Operating Income As Reported', 'Other Non Operating Income Expenses', 'Net Non Operating Interest Income Expense', 'Interest Expense Non Operating', 'Interest Income Non Operating', 'Operating Income', 'Operating Expense', 'Operating Revenue']
------------------------------
✅ MSFT의 'revenue' 관련 컬럼:
['Total Operating Income As Reported', 'Other Non Operating Income Expenses', 'Net Non Operating Interest Income Expense', 'Interest Expense Non Operating', 'Interest Income Non Operating', 'Operating Income', 'Operating Expense', 'Operating Revenue']
------------------------------
✅ GOOGL의 'revenue' 관련 컬럼:
['Total Operating Income As Reported', 'Other Non Operating Income Expenses', 'Net Non Operating Interest Income Expense', 'Interest Expense Non Operating', 'Interest Income Non Operating', 'Operating Income', 'Operating Expense', 'Operating Revenue']
------------------------------
✅ AMZN의 'revenue' 관련 컬럼:
['Total Operating Income As Reported', 'Other Non 

In [20]:
# 수정 내용: TotalLiabilitiesAndEquity > Total Debt

for ticker, data in financial_data.items():
    balance_sheet = data['balance_sheet']

    # .str.contains()를 이용해 'total'가 포함된 컬럼만 필터링
    current_columns = [col for col in balance_sheet.columns if 'total' in str(col).lower()]

    print(f"✅ {ticker}의 'total' 관련 컬럼:")
    print(current_columns)
    print("-" * 30)

✅ AAPL의 'total' 관련 컬럼:
['Total Debt', 'Total Capitalization', 'Total Equity Gross Minority Interest', 'Total Liabilities Net Minority Interest', 'Total Non Current Liabilities Net Minority Interest', 'Total Tax Payable', 'Total Assets', 'Total Non Current Assets']
------------------------------
✅ MSFT의 'total' 관련 컬럼:
['Total Debt', 'Total Capitalization', 'Total Equity Gross Minority Interest', 'Total Liabilities Net Minority Interest', 'Total Non Current Liabilities Net Minority Interest', 'Total Tax Payable', 'Total Assets', 'Total Non Current Assets']
------------------------------
✅ GOOGL의 'total' 관련 컬럼:
['Total Debt', 'Total Capitalization', 'Total Equity Gross Minority Interest', 'Total Liabilities Net Minority Interest', 'Total Non Current Liabilities Net Minority Interest', 'Total Tax Payable', 'Total Assets', 'Total Non Current Assets']
------------------------------
✅ AMZN의 'total' 관련 컬럼:
['Total Debt', 'Total Capitalization', 'Total Equity Gross Minority Interest', 'Total Li

In [21]:
# 수정 내용: CommonStockEquity > Common Stock Equity

for ticker, data in financial_data.items():
    balance_sheet = data['balance_sheet']

    # .str.contains()를 이용해 'equity'가 포함된 컬럼만 필터링
    current_columns = [col for col in balance_sheet.columns if 'equity' in str(col).lower()]

    print(f"✅ {ticker}의 'equity' 관련 컬럼:")
    print(current_columns)
    print("-" * 30)

✅ AAPL의 'equity' 관련 컬럼:
['Common Stock Equity', 'Total Equity Gross Minority Interest', 'Stockholders Equity', 'Other Equity Adjustments']
------------------------------
✅ MSFT의 'equity' 관련 컬럼:
['Common Stock Equity', 'Total Equity Gross Minority Interest', 'Stockholders Equity', 'Other Equity Adjustments', 'Long Term Equity Investment']
------------------------------
✅ GOOGL의 'equity' 관련 컬럼:
['Common Stock Equity', 'Total Equity Gross Minority Interest', 'Stockholders Equity', 'Other Equity Adjustments']
------------------------------
✅ AMZN의 'equity' 관련 컬럼:
['Common Stock Equity', 'Total Equity Gross Minority Interest', 'Stockholders Equity', 'Other Equity Adjustments']
------------------------------
✅ META의 'equity' 관련 컬럼:
['Common Stock Equity', 'Total Equity Gross Minority Interest', 'Stockholders Equity', 'Other Equity Adjustments', 'Long Term Equity Investment']
------------------------------


> **오류 2: `KeyError: 'Inventory'` 발생**
>
> 기업 META의 경우, 다음과 같은 이유로 재고자산이 존재하지 않기 때문에 오류가 발생했습니다.
>
> META(Meta Platforms, Inc.)와 같은 기술 기반 플랫폼 기업은 상품을 물리적으로 생산하거나 판매하지 않습니다. 이들은 주로 광고, 디지털 서비스 등을 통해 수익을 창출하기 때문에, 전통적인 제조업이나 유통업과는 달리 재고자산이 발생하지 않습니다. 따라서 META의 재무상태표에는 Inventory 계정이 존재하지 않는 것이 자연스러운 현상입니다.
>
> 따라서 당좌비율을 계산하기 위해 재고자산 금액을 0으로 간주하는 것이 회계적으로 가장 타당하다고 판단하여, 위의 코드를 수정하였습니다.
>
> 위의 코드는 이와 같은 **오류가 수정된 최종본**임을 알려드립니다. 다음의 과정은 META에서 Inventory 계정과목이 존재하지 않는지 여부를 확인하는 과정입니다.

In [27]:
for ticker, data in financial_data.items():
    balance_sheet = data['balance_sheet']

    # .str.contains()를 이용해 'inventory'가 포함된 컬럼만 필터링
    current_columns = [col for col in balance_sheet.columns if 'inventory' in str(col).lower()]

    print(f"✅ {ticker}의 'inventory' 관련 컬럼:")
    print(current_columns)
    print("-" * 30)

✅ AAPL의 'inventory' 관련 컬럼:
['Inventory']
------------------------------
✅ MSFT의 'inventory' 관련 컬럼:
['Inventory']
------------------------------
✅ GOOGL의 'inventory' 관련 컬럼:
['Inventory']
------------------------------
✅ AMZN의 'inventory' 관련 컬럼:
['Inventory']
------------------------------
✅ META의 'inventory' 관련 컬럼:
[]
------------------------------


> 위의 작업을 통해서 META에는 inventory 계정과목이 존재하지 않음을 확인했습니다. 물론, 기업 특성상 inventory가 존재하지 않는 것은 합리적인 경우이지만, 그럼에도 별도의 계정으로 존재하는 계정과목이 존재하는지를 추가적으로 확인하기 위해 아래와 같은 절차를 수행했습니다.

In [26]:
balance_sheet_aapl = financial_data['META']['balance_sheet']
print(balance_sheet_aapl.columns)

Index(['Treasury Shares Number', 'Ordinary Shares Number', 'Share Issued',
       'Total Debt', 'Tangible Book Value', 'Invested Capital',
       'Working Capital', 'Net Tangible Assets', 'Capital Lease Obligations',
       'Common Stock Equity', 'Total Capitalization',
       'Total Equity Gross Minority Interest', 'Stockholders Equity',
       'Gains Losses Not Affecting Retained Earnings',
       'Other Equity Adjustments', 'Retained Earnings',
       'Additional Paid In Capital', 'Capital Stock', 'Common Stock',
       'Total Liabilities Net Minority Interest',
       'Total Non Current Liabilities Net Minority Interest',
       'Other Non Current Liabilities', 'Tradeand Other Payables Non Current',
       'Long Term Debt And Capital Lease Obligation',
       'Long Term Capital Lease Obligation', 'Long Term Debt',
       'Current Liabilities', 'Other Current Liabilities',
       'Current Deferred Liabilities', 'Current Deferred Revenue',
       'Current Debt And Capital Lease Oblig

## **추가 학습: 프로젝트 진행 과정에서 마주친 핵심 파이썬 이론 총정리**

### **Q1. `[]`와 `[][]` 문법은 무엇인가요?**

**A.** 이 문법은 파이썬과 Pandas 라이브러리의 **데이터 선택(Indexing) 방식**에 대한 이해를 요구합니다. 단순한 요약을 넘어, 파이썬의 기본 자료형과 Pandas의 데이터 구조를 함께 이해해야 합니다.

* **파이썬 기본 이론**: 파이썬에서 `[]`는 **순서가 있는 자료형(Sequence)**에 접근할 때 사용됩니다. 대표적으로 **리스트**와 **딕셔너리**가 있죠.
    * **리스트**: `my_list = [10, 20, 30]`에서 `my_list[0]`은 첫 번째 요소인 `10`을 반환합니다.
    * **딕셔너리**: `my_dict = {'key': 'value'}`에서 `my_dict['key']`는 `'value'`를 반환합니다.

* **Pandas 적용**: Pandas의 `DataFrame`은 여러 열(Series)이 모인 2차원 구조입니다. `df[]` 문법은 딕셔너리와 유사하게 **열 이름(컬럼명)**을 키로 사용하여 해당 열을 선택합니다.
    * `df['컬럼명']`: 하나의 열을 선택하며, 결과는 **`Series`** 자료형입니다.
    * `df[['컬럼명1', '컬럼명2']]`: 두 번째 대괄호 `[]`는 파이썬의 **리스트** 문법입니다. 즉, Pandas에게 `['컬럼명1', '컬럼명2']`라는 **컬럼 리스트**를 전달하여, 이 리스트에 포함된 모든 열을 선택하라고 명령하는 것입니다. 결과는 **`DataFrame`** 자료형입니다.

* **중첩된 `[][]`**: `financial_data[ticker]['income_stmt']`는 **중첩된 딕셔너리 구조**에서 데이터를 찾아가는 과정입니다. 첫 번째 `[ticker]`는 `financial_data` 딕셔너리에서 특정 기업의 데이터를 담은 **내부 딕셔너리**를 반환합니다. 두 번째 `['income_stmt']`는 그 내부 딕셔너리에서 **손익계산서 데이터프레임**을 최종적으로 선택합니다.

### **Q2. 변수 할당과 `copy()`는 왜 함께 사용해야 하나요?**

**A.** 이 질문은 파이썬의 **객체(Object) 지향 프로그래밍** 기본 개념 중 '참조(Reference)'와 '복사(Copy)'에 대한 이해와 관련이 깊습니다.

* **파이썬 기본 이론 (참조)**: 파이썬에서 변수(`variable`)는 데이터가 저장된 **메모리 주소**를 가리키는 '이름표' 역할을 합니다. `df_a = df_b`라는 코드는 `df_a`와 `df_b`가 **동일한 메모리 주소**를 바라보게 합니다. 따라서 `df_a`를 수정하면 `df_b`도 함께 변경되는 '의도치 않은' 부작용이 발생할 수 있습니다.

* **`copy()`의 역할**: `.copy()` 메서드는 **원본 데이터와 분리된 새로운 메모리 공간**에 데이터를 복사합니다. `df_a = df_b.copy()`를 사용하면, `df_a`와 `df_b`는 서로 다른 메모리 주소를 가리키게 되어, `df_a`를 수정하더라도 `df_b`는 영향을 받지 않습니다. 데이터 분석에서 원본 데이터를 보호하고 독립적인 작업을 수행하기 위한 필수적인 방법입니다.

### **Q3. `for` 루프에서 `.items()`를 사용하는 이유는 무엇인가요?**

**A.** 이는 **딕셔너리를 효율적으로 순회하는 방법**에 대한 질문입니다.

* **파이썬 기본 이론**: 딕셔너리는 순서가 없는 자료형이지만, `for` 루프를 통해 모든 요소를 하나씩 반복할 수 있습니다. 딕셔너리를 순회하는 방법은 여러 가지가 있습니다.
    * `for key in my_dict`: 키(key)만 순회합니다. 값을 얻으려면 `my_dict[key]`와 같이 다시 접근해야 합니다.
    * `for value in my_dict.values()`: 값(value)만 순회합니다.
    * **`for key, value in my_dict.items()`**: **가장 효율적인 방법**입니다. `.items()` 메서드는 딕셔너리 내 모든 **`(키, 값)` 쌍**을 튜플 형태로 반환합니다. 이를 `for` 루프에 사용하면, 반복 한 번에 키와 값을 동시에 얻을 수 있어 코드가 훨씬 간결하고 빠릅니다.

### **Q4. `try-except`는 무엇인가요?**

**A.** 이는 **예외 처리(Exception Handling)**라는 프로그래밍 기법의 핵심입니다.

* **파이썬 기본 이론**: 코드가 실행되는 도중 오류가 발생하면, 파이썬은 프로그램의 실행을 멈추고 오류 메시지를 출력합니다. 이러한 '오류'를 '예외(Exception)'라고 부릅니다. `try-except` 구문은 이러한 예외가 발생하더라도 프로그램이 비정상적으로 종료되는 것을 막고, 지정된 방식으로 오류를 처리하게 해줍니다.
    * **`try` 블록**: 오류가 발생할 가능성이 있는 코드를 여기에 작성합니다.
    * **`except` 블록**: `try` 블록에서 예외가 발생했을 때 실행될 코드를 작성합니다. `as e`를 사용하여 발생한 예외 객체를 `e` 변수에 할당하면, 오류의 종류나 메시지를 확인할 수 있습니다.

* **실무 활용**: 데이터 수집 과정에서는 API 연결, 데이터 형식 등 다양한 이유로 오류가 발생할 수 있습니다. `try-except`는 이러한 오류를 미리 방지하여 프로그램이 안정적으로 작동하게 하고, 사용자에게 친절한 피드백을 제공합니다.

### **Q5. `income_stmt`와 `balance_sheet`는 임의의 약어인가요?**

**A.** 이 질문은 **데이터 분석 라이브러리와 회계 전문 용어의 관계**에 대한 이해를 돕습니다.

* **결론**: 이 용어들은 임의의 약어가 아니라, 금융 및 회계 업계에서 공공연하게 사용되는 **표준 용어**입니다.
    * **`income_stmt`**: **Income Statement(손익계산서)**의 줄임말입니다.
    * **`balance_sheet`**: **Balance Sheet(재무상태표)**의 정식 명칭입니다.

* **이론적 배경**: 전문 분야의 데이터를 다루는 라이브러리(예: `yfinance`)는 해당 분야의 전문가들이 이해하기 쉽도록 표준화된 용어를 사용합니다. 이는 코드의 가독성을 높이고, 개발자와 실무자 간의 소통을 원활하게 만듭니다.